In [1]:
# Mandatory cell for the rest of this assignment

%load_ext autoreload
%autoreload 2

from sys import path

path.append('./scripts')

In [2]:
from certificate import Certificate
from blockchain import Blockchain
from smart_contract import SmartContractDefinition, SmartContractWritingOperation
from network import Node
from node import BlockchainNode
from wallet import Wallet
from proof_of_stake import ProofOfStake
from block import Block
from pebble import Account
from time import sleep
from helpers import timestamp

We'll start by generating the wallets for supposedly the "users".
In our case, we'll also be giving each user an account (assuming they've registrated to the betting website).

The account also contains the amount of pebbles a user has.

In [8]:
walletNode1=Wallet()
walletNode2=Wallet()
walletNode3=Wallet()

Bob=Wallet()
BobAccount = Account(Bob.publicKey)
print('Bob has ', BobAccount.pebbleAmount, ' pebbles')
John=Wallet()
JohnAccount = Account(John.publicKey)
print('John has ', JohnAccount.pebbleAmount, ' pebbles')
Alice=Wallet()
AliceAccount = Account(Alice.publicKey)
print('Alice has ', AliceAccount.pebbleAmount, ' pebbles')
Emma=Wallet()
EmmaAccount = Account(Alice.publicKey)
print('Emma has ', EmmaAccount.pebbleAmount, ' pebbles')

pos = ProofOfStake(walletNode1.publicKey)

def reset_blockchain():
    Node.reset_network()
    global node1, node2, node3
    node1 = BlockchainNode(walletNode1, pos)
    node2 = BlockchainNode(walletNode2, pos)
    node3 = BlockchainNode(walletNode3, pos)

Bob has  20  pebbles
John has  20  pebbles
Alice has  20  pebbles
Emma has  20  pebbles


In [4]:
def read_text_file(fileName):
    with open(fileName, 'r', encoding='utf-8') as f:
        return f.read()
    
betFilePath="./scripts/bet.py"    

We'll start by assuming that Alice, a user, has generated a bet.
John sees the bet and wants to join in.

In [12]:
reset_blockchain()

bet1=SmartContractDefinition(Alice.publicKey,read_text_file(betFilePath))
Alice.sign(bet1)
node1.new_certificate(bet1)
sleep(1)



bet1_start = SmartContractWritingOperation(Alice.publicKey, bet1.hash(), 'startBet', [timestamp.now()+timestamp.now(), 'Lorum Lora', ['yes', 'no', 'maybe']])
Alice.sign(bet1_start)
node1.new_certificate(bet1_start)
sleep(1)



cert1 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert1)
node1.new_certificate(cert1)


cert2 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert2)
node1.new_certificate(cert2)


cert3 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert3)
node1.new_certificate(cert3)


cert4 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert4)
node1.new_certificate(cert4)


cert5 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert5)
node1.new_certificate(cert5)

cert6 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert6)
node1.new_certificate(cert6)


cert7 = SmartContractWritingOperation(John.publicKey, bet1.hash(), 'addBetter', [JohnAccount, 1, 1])
John.sign(cert7)
node1.new_certificate(cert7)

node2.blockchain.display()


{'Genesis Key': '00000000', 'Genesis Index': 0}
{'Public Key': '30820122', 'Signature': '31128778', 'timestamp': 1709044199972, 'type': <class 'block.Block'>}
{'Index': 1, 'Parent hash': 'a5347700'}
--Block certs--
{'Public Key': '30820122', 'Signature': '36d3ce93', 'timestamp': 1709044199898, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': '6063de7a', 'timestamp': 1709044198805, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': '3a2f4fc4', 'timestamp': 1709044199938, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': '9525a865', 'timestamp': 1709044197764, 'type': <class 'smart_contract.SmartContractDefinition'>}
{'Public Key': '30820122', 'Signature': '5815711d', 'timestamp': 1709044199854, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
----


In [34]:
print(read_text_file(betFilePath))

class SmartContract():
    def __init__(self,ownerPublicKey):
        self.ownerPublicKey=ownerPublicKey
        self.description=None
        self.startTime=None
        self.closingPeriod=None
        #dictionnary of betters with their public key, the amount staked, and the outcome they want
        self.betters=[]
        #list of strings for different outcomes
        self.outcomes=[]
        self.logs=[]
        self.type="BET"

#these two functions are to be called outside blockchain only_______________    
    def display(self,*args):
        print(self.description)
        for i,o in enumerate(self.outcomes):
            print(f'Select {i+1} for {o}')
        print(self.closingPeriod)
        print('Current betters---------')
        total={}
        for b in self.betters:
            print(f"User {b['PublicKey'][256:264]} bet {b['Amount']} on {b['Outcome']}.")
            if b['Outcome'] in total.keys():
                total[b['Outcome']]+=b['Amount']
            else:
      

Of course, each time John bets, the amount of pebbles he has should be decreased by the amount he used on the bet.

In [6]:
print('John has ', JohnAccount.pebbleAmount, ' pebbles')

John has  20  pebbles


In [7]:

cert_close = SmartContractWritingOperation(Alice.publicKey, bet1.hash(), 'closeBet', [1])
Alice.sign(cert_close)
node1.new_certificate(cert_close)

SmartContractObject = SmartContractDefinition.get_smart_contract_at_current_state(node1.blockchain, bet1.hash())
SmartContractObject.logDump()

SmartContractObject.display()

[node 3082.8da5.0001] Adding
{'Public Key': '30820122', 'Signature': '1755ab84', 'timestamp': 1709043503782, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
[node 3082.8da5.0001] I am sending the data "<block.Block object at 0x0000024E3FBFFBB0>" to EVERYBODY!
[node 3082.8da5.0001] I am sending the data "<block.Block object at 0x0000024E3FBFFBB0>" to "node 3082.5cd1.0001"
[node 3082.5cd1.0001] I am sending the data "<block.Block object at 0x0000024E3FBFFBB0>" to EVERYBODY!
[node 3082.5cd1.0001] I am sending the data "<block.Block object at 0x0000024E3FBFFBB0>" to "node 3082.8da5.0001"
[node 3082.8da5.0001] "node 3082.5cd1.0001" sent me a new a new block
[node 3082.5cd1.0001] I am sending the data "<block.Block object at 0x0000024E3FBFFBB0>" to "node 3082.e7f2.0001"
[node 3082.e7f2.0001] I am sending the data "<block.Block object at 0x0000024E3FBFFBB0>" to EVERYBODY!
[node 3082.e7f2.0001] I am sending the data "<block.Block object at 0x0000024E3FBFFBB0>" to "node 3082.8da

In [19]:
print('John has ', JohnAccount.pebbleAmount, ' pebbles')

John has  17  pebbles


In [20]:
node1.print_certbox()
node1.blockchain.display()

--CERTBOX 4 certs--
{'Public Key': '30820122', 'Signature': '9aa518f5', 'timestamp': 1709040312418, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': '1203745f', 'timestamp': 1709040312495, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': '06cc226c', 'timestamp': 1709040312532, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': '80fc268f', 'timestamp': 1709040312457, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
----
{'Genesis Key': '00000000', 'Genesis Index': 0}
{'Public Key': '30820122', 'Signature': '85db2e0f', 'timestamp': 1709040312374, 'type': <class 'block.Block'>}
{'Index': 1, 'Parent hash': 'e2d14cf4'}
--Block certs--
{'Public Key': '30820122', 'Signature': 'a04151f1', 'timestamp': 1709040312264, 'type': <class 'smart_contract.SmartContractWritingOperation'>}
{'Public Key': '30820122', 'Signature': 